In [1]:
import os
import json
import pandas as pd
import traceback

In [ ]:
%pip install langchain_community
%pip install langchain-openai

DEPRECATION: Loading egg at e:\genai\playground-openai\mcq-generator\.venv\lib\site-packages\aiohappyeyeballs-2.4.4-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\genai\playground-openai\mcq-generator\.venv\lib\site-packages\aiohttp-3.11.11-py3.12-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\genai\playground-openai\mcq-generator\.venv\lib\site-packages\aiosignal-1.3.2-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\genai\playground-op

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at e:\genai\playground-openai\mcq-generator\.venv\lib\site-packages\aiohappyeyeballs-2.4.4-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\genai\playground-openai\mcq-generator\.venv\lib\site-packages\aiohttp-3.11.11-py3.12-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\genai\playground-openai\mcq-generator\.venv\lib\site-packages\aiosignal-1.3.2-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at e:\genai\playground-op

In [3]:
from langchain_openai import ChatOpenAI

api_key=os.getenv("OPENAI_API_KEY");

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0.5,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key=api_key
)

In [4]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x00000247B055B170>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x00000247B057CE90>, root_client=<openai.OpenAI object at 0x00000247B0213320>, root_async_client=<openai.AsyncOpenAI object at 0x00000247B055B1D0>, model_name='gpt-4o', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'), max_retries=2)

In [5]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [8]:
RESPONSE_JSON = {
  "1": {
    "mcq": "multiple choice question goes here",
    "options": {
      "a": "choice goes here",
      "b": "choice goes here",
      "c": "choice goes here",
      "d": "choice goes here"
    },
    "answer": "correct answer option goes here"
  },
  "2": {
    "mcq": "multiple choice question goes here",
    "options": {
      "a": "choice goes here",
      "b": "choice goes here",
      "c": "choice goes here",
      "d": "choice goes here"
    },
    "answer": "correct answer option goes here"
  },
  "3": {
    "mcq": "multiple choice question goes here",
    "options": {
      "a": "choice goes here",
      "b": "choice goes here",
      "c": "choice goes here",
      "d": "choice goes here"
    },
    "answer": "correct answer option goes here"
  }
}

In [ ]:
TEMPLATE="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""